
# TFIDF and cosine similarity - toy example
#### Inspired by, and partly taken from the contributions of <a href="https://markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers/">Mark Needham</a>  and <a href="https://towardsdatascience.com/tf-idf-for-document-ranking-from-scratch-in-python-on-real-world-dataset-796d339a4089">William Scott</a>

### This Notebook demonstrate the use of TFIDF in retrieval. <br> ~6000 very short documents (stored in the papers/ directory) are read into memory, preprocessed to various degrees and are indexed for retrieval.<br> 
#### A toy of a toy (10 documents) are available in the directory <b>papers1/</b>

In [140]:

from IPython.display import HTML, display
import tabulate
import glob
#
corpus = [] # A list of tuples

i=0
for file in glob.glob("papers/*.txt"): #"papers1/*.txt" - 10 documents ...
    with open(file, "r") as paper:
#        filesfile.write(file[7:-4]+":  "+paper.read()+"\n")
        corpus.append((file, paper.read()))
        i+=1

#Define N, the number of documents
N=len(corpus)
print(corpus[0])


('papers/28418.txt', 'The complexity of perfect zero-knowledge')


In [141]:
def token_split(doc_or_query):
    tokens = doc_or_query.lower().split()
    processed_doc_or_query = []
    for w in tokens:

      
        if w not in stopwords.words("english"):
            processed_doc_or_query.append(w)
    return processed_doc_or_query

# Preprocessing
## we introduce preprocessing in two steps that use the nltk-package to different degrees
### 1. simple_preprocess() which only uses stop-words and punct. removal
### 2. preprocess():             here we can comment in / out different steps, to see the effect
### remember to call the correct function both for texts AND queries when experimenting with different preprocessing

### Define the simple preprocessing.

In [142]:
#### 1. simple_preprocess:
#### HERE WE ONLY IMPORT STOPWORDS LIST FROM NLTK, AND HANDLE PUNCTUATION

#### The nltk-package has a lot of useful tools for language technology.<br> 
from nltk.corpus import stopwords

symbols = r"!\"#$%&()*+-—.,/:;<=>?@[\]^_`{|}~"

# HERE WE USE THE STOPWORDS (NO Stemming, Lemmatization or any other stuff)
def simple_preprocess(doc_or_query):
    # returns a list of tokens
    txt = doc_or_query

    # REMOVE PUNCTUATION
    for ch in symbols:
        txt = txt.replace(ch, " ")  # re.sub(string.punctuation, " ", doc[1])
    return token_split(txt)
    # txt.lower() standardizes to low-case characters


### Define the more elaborate preprocessing

In [143]:
#### 2. preprocess:
#### MORE ELABORATE PREPROCESSING WHERE STEPS CAN BE SWITCHED OUT
#### BY COMMENTING OUT LINE

import preprocess as pp  # We import the python file preprocess.py with preprocessing function


def preprocess(doc_or_query):
    print("before:",doc_or_query)
    doc_or_query = pp.convert_lower_case(doc_or_query)
    
    doc_or_query = pp.remove_punctuation(
        doc_or_query
    )  # remove comma seperately
    
    doc_or_query = pp.remove_apostrophe(doc_or_query)
    doc_or_query = pp.remove_stop_words(doc_or_query)
    doc_or_query = pp.convert_numbers(doc_or_query)
    doc_or_query = pp.stemming(doc_or_query)
    doc_or_query = pp.remove_punctuation(doc_or_query)
    doc_or_query = pp.convert_numbers(doc_or_query)
    doc_or_query = pp.stemming(
        doc_or_query
    )  
    # needed again as we need to stem the words
    doc_or_query = pp.remove_punctuation(
        doc_or_query
    )  
    # needed again as num2word is giving few hypens and commas fourty-one
    doc_or_query = pp.remove_stop_words(
        doc_or_query
    )
    print("after:",doc_or_query)

    return token_split(doc_or_query)

In [144]:
import sys
import re
import numpy as np
import string
### aDF calculated in advance
symbols = r"!\"#$%&()*+-—.,/:;<=>?@[\]^_`{|}~"
  
DF = {}
c=0
processed_corpus=[]#An array of token arrays
ctr=0
for doc in corpus:
    processed_text=""
    txt=doc[1]
    processed_tokens=preprocess(txt)
    
    #DF includes actually our vocabulary, and for each word its global weight 
    for w in processed_tokens:
        try:
            # DF[w] is a set, and each document will only be added once.
            DF[w].add(ctr)
        except:
            DF[w] = {ctr}
                
    processed_corpus.append(processed_tokens)
    ctr += 1
print("ctr",ctr)
# At the end ctr = N

# WE only need the number of distinct documents indexed  by each word.
for j in DF:
    DF[j]=len(DF[j])

    #Print the first token array in processed_corpus
processed_corpus[0]
DF

before: The complexity of perfect zero-knowledge
after:  complex perfect zero knowledg
before: Lottery and stride scheduling: flexibile proportional-share resource management
after:  lotteri stride schedul flexibil proport share resourc manag
before: An investigation of documents from the World Wide Web
after:  investig document world wide web
before: Performance of multiversion and distributed two-phase locking concurrency control mechanisms in distributed databases
after:  perform multiv distribut two phase lock concurr control mechan distribut databa
before: Benchmarking Anomaly-Based Detection Systems
after:  benchmark anomali base detect system
before: Detecting causal relationships in distributed computations: in search of the holy grail
after:  detect causal relationship distribut comput search holi grail
before: Research problems in data warehousing
after:  research problem data wareh
before: Main Memory Database Systems: An Overview
after:  main memori databa system overview
b

before: On the encipherment of search trees and random access files
after:  enciph search tree random access file
before: THE MIT ALEWIFE MACHINE: A LARGE-SCALE DISTRIBUTED-MEMORY MULTIPROCESSOR
after:  mit alewif machin larg scale distribut memori multiprocessor
before: BASE: Using abstraction to improve fault tolerance
after:  base use abstract improv fault toler
before: A Software Platform for Testing Intrusion Detection Systems
after:  softwar platform test intru detect system
before: Embedded inodes and explicit grouping: exploiting disk bandwidth for small files
after:  emb inod explicit group exploit disk bandwidth small file
before: A decade of software model checking with SLAM
after:  decad softwar model check slam
before: Fast Replicated State Machines Over Partitionable Networks
after:  fast replic state machin partit network
before: Operating System Concepts, 4th Ed.
after:  oper system concept 4th ed
before: StackGuard: automatic adaptive detection and prevention of buffer

after:  unif base pointer analysi direct assign
before: Weak-consistency group communication and membership
after:  weak consist group commun membership
before: An Attack on the Last Two Rounds of MD4
after:  attack last two round md4
before: Free transactions with Rio Vista
after:  free transact rio vista
before: Using queries for distributed monitoring and forensics
after:  use queri distribut monitor foren
before: Knowledge and common knowledge in a distributed environment
after:  knowledg common knowledg distribut environ
before: Processor scheduling in shared memory multiprocessors
after:  processor schedul share memori multiprocessor
before: Removing backing store administration from the CAP operating system
after:  remov back store administr cap oper system
before: An Experimental Analysis of BGP Convergence Time
after:  experi analysi bgp converg time
before: Performance analysis and visualization
after:  perform analysi visual
before: Towards a Theory of Abstract Data Types: A

after:  gain effici transport servic appropri design implement choic
before: Protocol Verification as a Hardware Design Aid
after:  protocol verif hardwar design aid
before: The design, implementation and evaluation of SMART: a scheduler for multimedia applications
after:  design implement evalu smart schedul multimedia applic
before: A hybrid quarantine defense
after:  hybrid quarantin defen
before: Contention in shared memory algorithms
after:  content share memori algorithm
before: Life, death, and the critical transition: finding liveness bugs in systems code
after:  life death critic transit find live bug system code
before: The future of magnetic data storage technology
after:  futur magnet data storag technolog
before: Simulation-based `STRESS' Testing Case Study: A Multicast Routing Protocol
after:  simul base stress test case studi multicast rout protocol
before: An IPv6 Provider-Based Unicast Address Format
after:  ipv6 provid base unicast address format
before: Fine-grained 

after:  charact refer local www
before: Java Security: Present and Near Future
after:  java secur present near futur
before: Fast isolation of arbitrary forwarding faults
after:  fast isol arbitrari forward fault
before: Design and implementation of the Wisconsin storage system
after:  design implement wisconsin storag system
before: Cryptographic Computation: Secure Faut-Tolerant Protocols and the Public-Key Model
after:  cryptograph comput secur faut toler protocol public key model
before: Replicated distributed programs (fault tolerance, communication protocols, operating systems, remote procedure call, computer networks)
after:  replic distribut program fault toler commun protocol oper system remot procedur call comput network
before: Modeling concepts for VLSI CAD objects
after:  model concept vlsi cad object
before: A garbage-collecting typed assembly language
after:  garbag collect type assembl languag
before: Spawn: A Distributed Computational Economy
after:  spawn distribut co

...

after:  secret share infinit domain extend abstract
before: BGP Route Flap Damping
after:  bgp rout flap damp
before: Improved query performance with variant indexes
after:  improv queri perform variant index
before: Query Optimization in Database Systems
after:  queri optim databa system
before: Achieving scalability and expressiveness in an Internet-scale event notification service
after:  achiev scalabl express internet scale event notif servic
before: Simplifying fault-tolerance: providing the abstraction of crash failures
after:  simplifi fault toler provid abstract crash failur
before: Skew-aware automatic database partitioning in shared-nothing, parallel OLTP systems
after:  skew awar automat databa partit share noth parallel oltp system
before: Design & analysis of fault tolerant digital systems
after:  design analysi fault toler digit system
before: An evaluation of redundant arrays of disks using an Amdahl 5890
after:  evalu redund array disk use amdahl five thousand eight hu

after:  univ oper unari versu binari
before: The Cedar file system
after:  cedar file system
before: Distributed RAID - A New Multiple Copy Algorithm
after:  distribut raid new multipl copi algorithm
before: Correct memory operation of cache-based multiprocessors
after:  correct memori oper cach base multiprocessor
before: Reducing multidatabase query response time by tree balancing
after:  reduc multidataba queri respon time tree balanc
before: A Study of Index Structures for Main Memory Database Management Systems
after:  studi index structur main memori databa manag system
before: Real-time computing with lock-free shared objects
after:  real time comput lock free share object
before: Replication Techniques in Distributed Systems
after:  replic techniqu distribut system
before: Client-Server Paradise
after:  client server paradi
before: Holding intruders accountable on the Internet
after:  hold intrud account internet
before: Operating system protection for fine-grained programs
aft

after:  exploit punctuat semant continu data stream
before: IRM Enforcement of Java Stack Inspection
after:  irm enforc java stack inspect
before: A universal access smart-card-based secure file system
after:  univ access smart card base secur file system
before: Design and Implementation of a Distributed X-Multiplexor
after:  design implement distribut multiplexor
before: Main memory database recovery
after:  main memori databa recoveri
before: Coherent functions and program checkers
after:  coher function program checker
before: The Smalltalk-76 programming system design and implementation
after:  smalltalk seventi six program system design implement
before: A scalable content-addressable network
after:  scalabl content address network
before: An Improvement of the Fiat-Shamir Identification and Signature Scheme
after:  improv fiat shamir identif signatur scheme
before: Asynchronous fault-tolerant total ordering algorithms
after:  asynchron fault toler total order algorithm
before: A

after:  framework interprocedur optim presenc dynam class load
before: Query processing in main memory database management systems
after:  queri process main memori databa manag system
before: Economical solutions for the critical section problem in a distributed system (Extended Abstract)
after:  econom solut critic section problem distribut system extend abstract
before: Design and evaluation of a continuous consistency model for replicated services
after:  design evalu continu consist model replic servic
before: Combinatorics: set systems, hypergraphs, families of vectors, and combinatorial probability
after:  combin set system hypergraph famili vector combinatori probabl
before: An adaptive quality of service aware middleware for replicated services
after:  adapt qualiti servic awar middlewar replic servic
before: Concurrent Timestamping Made Simple
after:  concurr timestamp made simpl
before: Towards a theory of software protection and simulation by oblivious RAMs
after:  toward t

{'complex': 56,
 'perfect': 4,
 'zero': 21,
 'knowledg': 56,
 'lotteri': 2,
 'stride': 3,
 'schedul': 61,
 'flexibil': 1,
 'proport': 5,
 'share': 202,
 'resourc': 70,
 'manag': 200,
 'investig': 5,
 'document': 15,
 'world': 39,
 'wide': 62,
 'web': 80,
 'perform': 310,
 'multiv': 11,
 'distribut': 670,
 'two': 43,
 'phase': 13,
 'lock': 50,
 'concurr': 182,
 'control': 189,
 'mechan': 67,
 'databa': 362,
 'benchmark': 37,
 'anomali': 10,
 'base': 259,
 'detect': 76,
 'system': 1156,
 'causal': 22,
 'relationship': 7,
 'comput': 344,
 'search': 42,
 'holi': 1,
 'grail': 1,
 'research': 15,
 'problem': 97,
 'data': 327,
 'wareh': 8,
 'main': 14,
 'memori': 174,
 'overview': 30,
 'valuat': 1,
 'trust': 26,
 'open': 21,
 'network': 438,
 'reput': 9,
 'approach': 131,
 'choo': 4,
 'reliabl': 111,
 'peer': 59,
 'design': 172,
 'computer—th': 1,
 'six': 6,
 'thousand': 20,
 'hundr': 37,
 'fail': 14,
 'stop': 7,
 'faulti': 12,
 'program': 274,
 'empir': 15,
 'studi': 59,
 'competitv': 1,
 's

# We Form the TFIDF valued TD-matrix

In [145]:
from collections import Counter

doc = 0
tf_idf = {}  # Initializing the matrix
showed = {}
for d in range(N):  # For all documents
    tokens = processed_corpus[d]
    counter = Counter(tokens)  # counts unique tokens in the tokens array
    # and creates a dictionary of tokens counts
    words_count = len(tokens)
    for token in np.unique(tokens):  # sorted unique tokens
        tf = counter[token]  # /words_count
        logtf = 1 + np.log2(tf)  # log
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log2(1 + (N / df))  # log

        tf_idf[d, token] = (
            logtf * idf
        )  # tf_idf is implemented as tuple-keyed dictionary
doc += 1
# Printing an example from the Matrix: the TFIDF value of the word "Automated" in document 1
print(tf_idf)

{(0, 'complex'): 6.708591257051913, (0, 'knowledg'): 6.708591257051913, (0, 'perfect'): 10.50308035175293, (0, 'zero'): 8.114981701344005, (1, 'flexibil'): 12.502334580197997, (1, 'lotteri'): 11.502583213553354, (1, 'manag'): 4.9071310247467945, (1, 'proport'): 10.1814007617393, (1, 'resourc'): 6.3901075244029935, (1, 'schedul'): 6.586439915126655, (1, 'share'): 4.893256469875102, (1, 'stride'): 10.917869303345588, (2, 'document'): 8.598920958311076, (2, 'investig'): 10.1814007617393, (2, 'web'): 6.199917680274569, (2, 'wide'): 6.563227031009598, (2, 'world'): 7.2263504349051315, (3, 'concurr'): 5.038858707456259, (3, 'control'): 4.986097863455075, (3, 'databa'): 4.0895711886854045, (3, 'distribut'): 6.543511353072951, (3, 'lock'): 6.870611313044545, (3, 'mechan'): 6.452563965215617, (3, 'multiv'): 9.045387368995543, (3, 'perform'): 4.301068360412807, (3, 'phase'): 8.804875637995146, (3, 'two'): 7.086475708101401, (4, 'anomali'): 9.182642644437047, (4, 'base'): 4.5482937906737595, (4, 

# Simple matching of a query 
## add up term - TFIDF scores for each doc

In [146]:
from collections import deque

outtable_simple = []


def matching_score(query):
    # Process the query just like you processed the documents
    # remove punctuation
    processed_tokens = preprocess(query)

    tokens = processed_tokens
    print("Matching Score")
    print("\nQuery:", query)
    print("tokens:")
    print(tokens)

    query_weights = {}
    # Simply add up the tfidfs for the words in the documents they index.
    for key in tf_idf:
        # remember, key is composed of
        if key[1] in tokens:
            try:
                print("key[1]", key[1])
                if key[0] not in query_weights:
                    query_weights[key[0]]=0.0
                query_weights[key[0]] += tf_idf[key]  # Accummulate and add tfidf-values for the term in each document
            except:
                print("exception:", key, tf_idf[key])
                query_weights[key[0]] = tf_idf[key]  #
            # print("query_weights["+str(key[0])+"]="+str(query_weights[key[0]]))
    # Sort the resulting weights to give us a ranked list
    query_weights = sorted(query_weights.items(), key=lambda x: x[1], reverse=True)

    # print("query_weights", query_weights)

    l = []
    qw = []
    # List the first 10 matching documents
    for i in query_weights[:10]:
        l.append(i[0])
        qw.append(i[1])
    queue = deque(qw)
    outtable_simple.append(["Query: ", "'"+query+"'", ""])
    outtable_simple.append(["doc_nr", "doc", "score"])
    for d in l:
        score = queue.popleft()
        outtable_simple.append([d, corpus[d], score])
        # print(d,corpus[d], score)

In [147]:
#matching_score("Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")
query="Operating system"
matching_score(query)

display(HTML(tabulate.tabulate(outtable_simple, tablefmt='html')))


before: Operating system
after:  oper system
Matching Score

Query: Operating system
tokens:
['oper', 'system']
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
ke

key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] system
key[1] system
key[1] system
key[1] oper
key[1] sys

Query:,'Operating system',
doc_nr,doc,score
4161,"('papers/1098638.txt', 'The Cambridge CAP computer and its operating system (Operating and programming systems series)')",15.256365260645481
1315,"('papers/1268686.txt', 'Extending the operating system at the user level: the Ufo global file system')",10.21750655318922
1330,"('papers/168629.txt', 'The impact of operating system structure on memory system performance')",10.21750655318922
1516,"('papers/121911.txt', 'LOCUS operating system, a transparent system')",10.21750655318922
4976,"('papers/325090.txt', 'Pros and cons of operating system transactions for data base systems')",10.21750655318922
72,"('papers/822430.txt', 'Operating System Directions for the Next Millennium')",7.6281826303227405
84,"('papers/562353.txt', 'Operating System Concepts, 4th Ed.')",7.6281826303227405
110,"('papers/808449.txt', 'On attaining reliable software for a secure operating system')",7.6281826303227405
162,"('papers/850712.txt', 'Removing backing store administration from the CAP operating system')",7.6281826303227405


In [148]:
# Using the numpy.linalg package to multiply the lengths of the vectors
def cosine_sim(a, b):
    cos_sim = np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))
    return cos_sim

# creating a TD numpy-matrix D, with tfidf-values
### For mathematical calculations, it is much better to use the numpy-package.<br> For this we need to reform the tf_idf matrix into a numpy matrix. We call it D

In [149]:
total_vocab = [x for x in DF]
total_vocab_size = len(DF)

print()
D = np.zeros((N, total_vocab_size))
for tpl in tf_idf:  # tpl is a tuple (tpl[0]: document number tpl[1] term)
    try:
        ind = total_vocab.index(tpl[1])
        D[tpl[0]][ind] = tf_idf[tpl]
    except:
        print("passed")
        pass
print(tpl)


(5800, 'process')


# generating a vector of tokens 
##  for example a query vector
### This vector can be "cosined" with all the document vectors,<br> to get the similarities, and rank by them.

In [150]:
import math


def gen_vector(tokens):
    # We generate a vector of tfidf values the vocabulary from the keys of the DF dictionary
    total_vocab = [x for x in DF]
    print(total_vocab)
    Q = np.zeros((len(total_vocab)))

    counter = Counter(tokens)
    words_count = len(tokens)

    query_weights = {}

    for token in np.unique(tokens):
        """
        tf = (
            counter[token] / words_count
        )  # The local weight: frequency of the token in the vector

        # not all query vectors are represented in the vocabulary
        if token in DF:
            df = DF[token]  # DF is the global weight of the term
        else:
            df = 0
        idf = math.log2((N + 1) / (df + 1))  # log
        """
        tf = counter[token]  # /words_count
        logtf = 1 + np.log2(tf)  # log
        if token in DF:
            df = DF[token]
        else:
            df = 0
        idf = np.log2(1 + (N / df))  # log

        try:
            ind = total_vocab.index(token)
            Q[ind] = tf * idf
        except:
            pass
    return Q

In [154]:
outtable_cos = []


def cosine_similarity(query, D=D):
    # Create an array of cosine values
    print("Cosine Similarity")
    preprocessed_query = preprocess(query)
    # tokens = word_tokenize(str(preprocessed_query))
    tokens = preprocessed_query
    print("\nQuery:", query)
    print("")
    print(tokens)
    # print("D=", type(D))

    d_cosines = []

    query_vector = gen_vector(tokens)
    for q in query_vector:
        print(q)
    # We go through all vectors in the TD (tfidf) matrix D
    for d in D:
        cs = cosine_sim(query_vector, d)
        if np.isnan(cs):
            cs = np.float_(-10e3)
        d_cosines.append(cs)

    # argsort() returns the indexes that would sort the array.
    ## sorts by the cosines, but returns the indexes (document numbers, the first 10.)
  
    out = np.array(d_cosines).argsort()[-10:][::-1]
    outtable_cos.append(["Query: ", "'"+query+"'", ""])
    outtable_cos.append(["doc_nr", "doc", "score"])

    for d in out:
        outtable_cos.append([d, corpus[d], d_cosines[d]])

In [155]:
# cosine_similarity("Without the drive of Rebeccah's insistence, Kate lost her momentum. She stood next a slatted oak bench, canisters still clutched, surveying")
cosine_similarity(query)

Cosine Similarity
before: Operating system
after:  oper system

Query: Operating system

['oper', 'system']
['complex', 'perfect', 'zero', 'knowledg', 'lotteri', 'stride', 'schedul', 'flexibil', 'proport', 'share', 'resourc', 'manag', 'investig', 'document', 'world', 'wide', 'web', 'perform', 'multiv', 'distribut', 'two', 'phase', 'lock', 'concurr', 'control', 'mechan', 'databa', 'benchmark', 'anomali', 'base', 'detect', 'system', 'causal', 'relationship', 'comput', 'search', 'holi', 'grail', 'research', 'problem', 'data', 'wareh', 'main', 'memori', 'overview', 'valuat', 'trust', 'open', 'network', 'reput', 'approach', 'choo', 'reliabl', 'peer', 'design', 'computer—th', 'six', 'thousand', 'hundr', 'fail', 'stop', 'faulti', 'program', 'empir', 'studi', 'competitv', 'spin', 'multiprocessor', 'bug', 'deviant', 'behavior', 'gener', 'infer', 'error', 'code', 'structur', 'parallel', 'algorithm', 'step', 'toward', 'cach', 'resid', 'transact', 'process', 'risc', 'group', 'logic', 'vs', 'physic

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.09596087243655117
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [6.70859126 0.         0.         ... 0.         0.         0.        ]
cs 0.09122187017797162
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.17931171402894036
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
c

cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [6.70859126 0.         0.         ... 0.         0.         0.        ]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.

cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.06186985138574436
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.04890443828818333
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.059029755477478484
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0

...

query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.32913461936671257
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.09507879286866525
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.07495667518989506
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0.

query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.060573333052081006
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.53355

query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 

d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.272349143141926
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.05604252193999883
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.05288055944292481
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.12599788633800194
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs

query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.37038004958195936
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.06305367875542216
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0. 0.]
cs 0.0
query_vector [0. 0. 0. ... 0. 0. 0.]
d [0. 0. 0. ... 0. 0.

In [156]:
from IPython.display import HTML, display
import tabulate


display(HTML(tabulate.tabulate(outtable_simple, tablefmt="html")))
display(HTML(tabulate.tabulate(outtable_cos, tablefmt="html")))

Query:,'Operating system',
doc_nr,doc,score
4161,"('papers/1098638.txt', 'The Cambridge CAP computer and its operating system (Operating and programming systems series)')",15.256365260645481
1315,"('papers/1268686.txt', 'Extending the operating system at the user level: the Ufo global file system')",10.21750655318922
1330,"('papers/168629.txt', 'The impact of operating system structure on memory system performance')",10.21750655318922
1516,"('papers/121911.txt', 'LOCUS operating system, a transparent system')",10.21750655318922
4976,"('papers/325090.txt', 'Pros and cons of operating system transactions for data base systems')",10.21750655318922
72,"('papers/822430.txt', 'Operating System Directions for the Next Millennium')",7.6281826303227405
84,"('papers/562353.txt', 'Operating System Concepts, 4th Ed.')",7.6281826303227405
110,"('papers/808449.txt', 'On attaining reliable software for a secure operating system')",7.6281826303227405
162,"('papers/850712.txt', 'Removing backing store administration from the CAP operating system')",7.6281826303227405


Query:,'Operating system',
doc_nr,doc,score
5740,"('papers/6074.txt', 'Distributed operating systems')",0.865963145630514
3252,"('papers/184674.txt', 'Distributed operating systems')",0.865963145630514
3989,"('papers/549055.txt', 'Distributed Operating Systems and Algorithms')",0.7159835959134231
2025,"('papers/504478.txt', 'Server operating systems')",0.6910173342475195
4050,"('papers/574894.txt', 'Operating Systems Theory')",0.6572814338161644
3685,"('papers/906270.txt', 'Correctness in operating systems')",0.6129987513476209
5734,"('papers/356680.txt', 'Fault Tolerant Operating Systems')",0.6108244460923834
5398,"('papers/21853.txt', 'Operating systems: design and implementation')",0.6098756273100501
2514,"('papers/540365.txt', 'Operating system principles')",0.5975157817870321
